In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as la
from scipy.integrate import quad
import math
import statsmodels.api as sm
from math import e

In [3]:
dataint = pd.read_csv(r'C:\Users\Azri\Desktop\diabetes2.csv')
dataint = sm.add_constant(dataint)
dataint.head(3)

,const,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.0,6,148,72,35,0,33.6,0.627,50,1
1,1.0,1,85,66,29,0,26.6,0.351,31,0
2,1.0,8,183,64,0,0,23.3,0.672,32,1


In [4]:
#TFN for Low Risk
LL = 0.01
LM = 0.03
LR = 0.05

#TFN for High Risk
HL = 0.95
HM = 0.97
HR = 0.99

data={'0': dataint.loc[dataint['Outcome']==0].drop('Outcome',axis=1),
      '1':dataint.loc[dataint['Outcome']==1].drop('Outcome',axis=1),
      'Y':dataint['Outcome'],
      'X':dataint.drop('Outcome',axis=1)}

def fuzzy_ZK(LL,LM,LR,HL,HM,HR):
    #Spread
    s = LM-LL
    f = lambda α: α*(math.log((HM-(s)*(1-α))/(1-HM+(s)*(1-α))))
    integralf = quad(f,0,1)
    g = lambda α: α*(math.log((HM+(s)*(1-α))/(1-HM-(s)*(1-α))))
    integralg = quad(g,0,1)
    zhigh = integralg[0] + integralf[0]

    h = lambda α: 2*α*(1-α)*(math.log((HM-(s)*(1-α))/(1-HM+(s)*(1-α))))
    integralh = quad(h,0,1)
    j = lambda α: 4*α*(1-α)**2
    integralj = quad(j,0,1)
    k = lambda α: 2*α*(1-α)*(math.log((HM+(s)*(1-α))/(1-HM-(s)*(1-α))))
    integralk = quad(k,0,1)
    khigh = (integralk[0]-integralh[0])*3
    
    f = lambda α: α*(math.log((LM-(s)*(1-α))/(1-LM+(s)*(1-α))))
    integralf = quad(f,0,1)
    g = lambda α: α*(math.log((LM+(s)*(1-α))/(1-LM-(s)*(1-α))))
    integralg = quad(g,0,1)
    zlow = integralg[0] + integralf[0]
    
    h = lambda α: 2*α*(1-α)*(math.log((LM-(s)*(1-α))/(1-LM+(s)*(1-α))))
    integralh = quad(h,0,1)
    i = lambda α: 2*α*(1-α)*(math.log((LM+(s)*(1-α))/(1-LM-(s)*(1-α))))
    integrali = quad(i,0,1)
    klow = (integrali[0]-integralh[0])*3
    
    ZK = {'ZHigh':zhigh,
          'KHigh':khigh,
          'ZLow':zlow,
          'KLow':klow}
    
    return ZK
    

In [5]:
ZK = fuzzy_ZK(LL,LM,LR,HL,HM,HR)

In [6]:
ZK

{'ZHigh': 3.5170597215640518,
 'KHigh': 0.733911717182183,
 'ZLow': -3.5170597215640527,
 'KLow': 0.7339117171821843}

In [7]:
def fuzzy_parameters_est(ZKlist,data):
    def fuzzy_sse(ZKlist,data):
        Za = data['1']*ZK['ZHigh']
        Zb = data['0']*ZK['ZLow']
        Z = pd.concat([Za,Zb])
        Z = Z.sum().values
        Ka = data['1']*ZK['KHigh']
        Kb = data['0']*ZK['KLow']
        K = pd.concat([Ka,Kb])
        K = K.sum().values
        ZKList = {'Z':Z,'K':K}
        return ZKList
    ZKList = fuzzy_sse(ZK,data)
    Z = ZKList['Z']
    K = ZKList['K']
    X = data['X'].values
    XT = X.transpose()
    A = XT@X
    Ainv = np.linalg.inv(A)
    a = Ainv @ Z
    s = Ainv @ K
    s[s<0.0000001] = 0
    Parameters = pd.DataFrame()
    Parameters['Variables'] = data['X'].columns
    Parameters['a'] = a
    Parameters['s'] = s
    return Parameters
Parameters = fuzzy_parameters_est(ZK,data)
Parameters

,Ratios,a,s
0,const,-9.523454,0.733912
1,Pregnancies,0.144846,0.000000
2,Glucose,0.041644,0.000000
3,BloodPressure,-0.016403,0.000000
4,SkinThickness,0.001087,0.000000
5,Insulin,-0.001270,0.000000
6,BMI,0.093160,0.000000
7,DiabetesPedigreeFunction,1.035686,0.000000
8,Age,0.018439,0.000000


In [14]:
def fuzzy_log_transform(Parameters,ata):
    a = Parameters['a'].values
    s = Parameters['s'].values
    X = data['X'].values
    log_poss_a = X@a
    log_poss_s = X@s
    poss_l = e**(log_poss_a-log_poss_s)/(e**(log_poss_a-log_poss_s)+1)
    poss_a = e**(log_poss_a)/(e**(log_poss_a)+1)
    poss_r = e**(log_poss_a+log_poss_s)/(e**(log_poss_a+log_poss_s)+1)
    poss = pd.DataFrame()
    poss['l'] = poss_l
    poss['a'] = poss_a
    poss['r'] = poss_r
    return poss
poss = fuzzy_log_transform(Parameters,data)
poss

,l,a,r
0,0.582627,0.744117,0.858318
1,0.014620,0.029981,0.060493
2,0.716898,0.840645,0.916594
3,0.012067,0.024813,0.050337
4,0.833381,0.912432,0.955959
...,...,...,...
763,0.164908,0.291473,0.461494
764,0.150885,0.270169,0.435399
765,0.072781,0.140538,0.254091
766,0.139754,0.252859,0.413501


In [22]:
for index, row in poss.iterrows():
    m1 = (1-0)/(row['a']-row['l'])
    m2 = (0-1)/(row['r']-row['a'])
    c1 = (m1*(-row['l']))
    c2 = (m2*(-row['a']))+1
    integralh = quad(h,row['l'],row['a'])
    i = lambda α: m2*(α**2) + c2*α
    integrali = quad(i,row['a'],row['r'])
    j = lambda α: m1*α + c1
    integralj = quad(j,row['l'],row['a'])
    k = lambda α: m2*α + c2
    integralk = quad(k,row['a'],row['r'])
    poss.at[index, 'defuzz'] = (integralh[0]+integrali[0])/(integralj[0]+integralk[0])
poss

,l,a,r,defuzz,Performance,Crisp
0,0.582627,0.744117,0.858318,0.728354,1,1
1,0.014620,0.029981,0.060493,0.035032,0,0
2,0.716898,0.840645,0.916594,0.824713,1,1
3,0.012067,0.024813,0.050337,0.029072,0,0
4,0.833381,0.912432,0.955959,0.900591,1,1
...,...,...,...,...,...,...
763,0.164908,0.291473,0.461494,0.305959,0,0
764,0.150885,0.270169,0.435399,0.285484,0,0
765,0.072781,0.140538,0.254091,0.155803,0,0
766,0.139754,0.252859,0.413501,0.268705,1,0


In [52]:
poss['Performance'] = data['Y']
results = []
threshold = [0.3,0.4,0.5,0.6,0.7]
for threshold in threshold:
    poss['Crisp'] = (poss['defuzz'] >= threshold).astype(int)
    TP = len(poss[(poss['Performance'] == 1) & (poss['Crisp'] == 1)])
    TN = len(poss[(poss['Performance'] == 0) & (poss['Crisp'] == 0)])
    FP = len(poss[(poss['Performance'] == 0) & (poss['Crisp'] == 1)])
    FN = len(poss[(poss['Performance'] == 1) & (poss['Crisp'] == 0)])
    acc = (TP+TN)/len(data['Y'])
    sens = TP/(TP + FN)
    spec = TN/(TN + FP)
    results.append((acc, sens, spec))
results = pd.DataFrame(results,
                       columns={'Accuracy','Sensitivity','Specificity'},
                       index=['0.3','0.4','0.5','0.6','0.7'])

In [53]:
results

,Accuracy,Sensitivity,Specificity
0.3,0.756510,0.764925,0.752
0.4,0.757812,0.630597,0.826
0.5,0.783854,0.578358,0.894
0.6,0.776042,0.496269,0.926
0.7,0.755208,0.376866,0.958
